In [1]:
import plotly.express as px
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.graph_objects as go


In [2]:
rjPath = "../RJdata/RJ.json"
dengueDataPath = "../RJdata/mainRJData.csv"

dengueData = pd.read_csv(dengueDataPath)

In [3]:
fig = go.Figure(data=go.Choropleth(
    featureidkey="properties.NOME",
    locationmode = 'geojson-id',
    locations=dengueData["Municipios"],
    z = dengueData["2010"],
    autocolorscale=False,
    colorscale="Rainbow",
    geojson=rjPath
))

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

# Trying to create frames

In [4]:
weeklyDengueDataPath = "../RJdata/Dengue_Brasil_2010-2016_Daniel.xlsx"

names = ['id', 'cities']
for i in range(1, 53):
    names.append(str(i))
names.append('total')


weeklyDengueData = pd.read_excel(weeklyDengueDataPath, names=names, header=None, sheet_name=None)

weeklyDengueData["2010"].head()


,id,cities,1,2,3,4,5,6,7,8,...,44,45,46,47,48,49,50,51,52,total
0,330010,Angra dos Reis,6,4,0,2,3,3,7,12,...,9,6,1,4,4,0,9,9,0,888
1,330015,Aperibé,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,16
2,330020,Araruama,0,0,0,3,4,2,4,8,...,12,1,0,0,0,0,2,1,0,629
3,330022,Areal,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,330023,Armação dos Búzios,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,33


## Usas o plotly express para fazer os dados

Cada cidade deverá ter uma linha com o valor atualizado a cada semana

cidade - semana - dados

angra  -    1   -    20

angra  -    2   -    30

angra  -   ...  -   ...

angra  -   52   -   888

rio    -    1   -   0

....   

In [5]:
yearToUse = "2010"

figDict = dict(
    data=[],
    layout=dict(),
    frames=[]
)

slider = dict(
    active=0,
    yanchor="top",
    xanchor="left",
    currentvalue=dict(
        font=dict(size=20),
        prefix="Semana: ",
        visible=True,
        xanchor="right"
    ),
    transition=dict(duration=300, easing="cubic-in-out"),
    pad=dict(b=10,t=20),
    len=0.9,
    x=0.1,
    y=0,
    steps=[]
)

titleStr = f"Casos de dengue ao longo do ano de {yearToUse}."
figDict["layout"]["xaxis"] = dict(showgrid=False, zeroline=False,showticklabels=False)
figDict["layout"]["yaxis"] = dict(showgrid=False, zeroline=False, showticklabels=False)
figDict["layout"]["hovermode"] = "closest"
figDict["layout"]["showlegend"] = False
figDict["layout"]["title"] = titleStr

playButton = dict(
    args= [
        None,
        dict(
            frame=dict(duration=250, redraw=False),
            fromcurrent=True,
            transition=dict(duration=300, easing= "quadratic-in-out")
        )
    ],
    label="Play",
    method="animate"
)

pauseButton = dict(
    args= [
        [None],
        dict(
            frame=dict(duration=0, redraw=False),
            mode="immediate",
            transition=dict(duration=0)
        )
    ],
    label="Pause",
    method="animate"
)

figDict["layout"]["updatemenus"] = [
    dict(
        buttons=[playButton, pauseButton],
        direction="left",
        pad=dict(r=10, t=87),
        showactive=True,
        type="buttons",
        x=0.1,
        xanchor="right",
        y=0,
        yanchor="top"
    )
]

In [6]:
weeks = np.linspace(1,52,52,dtype=int)

weeklyDengueData[yearToUse]["Cumulative"] = np.zeros(92)

weeksDict = {}
for week in weeks:
    #Get variables from the week
    weeklyDengueData[yearToUse]["Cumulative"] = weeklyDengueData[yearToUse]["Cumulative"] + weeklyDengueData[yearToUse][str(week)]
    data = go.Choropleth(
        featureidkey="properties.NOME",
        locationmode = 'geojson-id',
        locations=weeklyDengueData[yearToUse]["cities"],
        z = weeklyDengueData[yearToUse]["Cumulative"],
        geojson=rjPath,
        colorscale = 'Rainbow'
    )
    weeksDict[week] = data

figDict["data"] = weeksDict[1]

for week in weeks:
    frame = dict(
        data=data,
        name=str(week)
    )
    figDict["frames"].append(frame)
    slider_step = dict(
        args = [
            [week+1],
            dict(
                frame=dict(duration=300, redraw=False),
                mode="immediate",
                transition=dict(duration=300)
            )
        ],
        label=str(week+1),
        method="animate"
    )
    slider["steps"].append(slider_step)


figDict["layout"]["sliders"] = [slider]
print(figDict)

{'data': Choropleth({
    'colorscale': [[0.0, 'rgb(150,0,90)'], [0.125, 'rgb(0,0,200)'], [0.25,
                   'rgb(0,25,255)'], [0.375, 'rgb(0,152,255)'], [0.5,
                   'rgb(44,255,150)'], [0.625, 'rgb(151,255,0)'], [0.75,
                   'rgb(255,234,0)'], [0.875, 'rgb(255,111,0)'], [1.0,
                   'rgb(255,0,0)']],
    'featureidkey': 'properties.NOME',
    'geojson': '../RJdata/RJ.json',
    'locationmode': 'geojson-id',
    'locations': array(['Angra dos Reis', 'Aperibé', 'Araruama', 'Areal', 'Armação dos Búzios',
                        'Arraial do Cabo', 'Barra do Piraí', 'Barra Mansa', 'Belford Roxo',
                        'Bom Jardim', 'Bom Jesus do Itabapoana', 'Cabo Frio',
                        'Cachoeiras de Macacu', 'Cambuci', 'Campos dos Goytacazes', 'Cantagalo',
                        'Carapebus', 'Cardoso Moreira', 'Carmo', 'Casimiro de Abreu',
                        'Comendador Levy Gasparian', 'Conceição de Macabu', 'Cordeiro',
      

In [7]:
fig = go.Figure(figDict)

fig.update_geos(
    fitbounds="locations",
    visible=False
)

fig.show()

In [37]:
dataDFPath = "./animationPlotlyOutput/2010/2010_out.shp"

dataDF = weeklyDengueData["2010"]
dataDF["Cumulative"] = 0

# print(dataDF)

mainDataList = []
for week in weeks:
    for i in range(len(dataDF["cities"])):
        dataDF["Cumulative"][i] += dataDF[str(week)][i]
        dataList = [ dataDF["cities"][i], week, dataDF["Cumulative"][i] ]
        mainDataList.append(dataList)
        
cols = ["Municipio", "timeframe", "Value"]

finalDF = pd.DataFrame(mainDataList, columns=cols)

print(finalDF)



/tmp/ipykernel_46897/3709310513.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



               Municipio  timeframe  Value
0         Angra dos Reis          1      6
1                Aperibé          1      0
2               Araruama          1      0
3                  Areal          1      0
4     Armação dos Búzios          1      0
...                  ...        ...    ...
4779           Três Rios         52     11
4780             Valença         52    500
4781           Varre-Sai         52      3
4782           Vassouras         52    321
4783       Volta Redonda         52    161

[4784 rows x 3 columns]


In [38]:
fig = px.choropleth(
    finalDF,
    locations="Municipio",
    geojson=rjPath,
    featureidkey="properties.NOME",
    locationmode = 'geojson-id',
    animation_frame="timeframe",
    color="Value"
)

fig.update_geos(
    fitbounds="locations",
    visible=False
)

fig.show()